In [1]:
import pandas as pd
import numpy as np
import torch
import warnings
import sys, os
import json

sys.path.append(os.path.abspath('..'))

ais_type_labels = '../../data/ais_type_labels.csv'
cleaned_detections_path = '../../data/cleaned_data/preprocessed_radar_detections.csv'
ais_type_labels = pd.read_csv(ais_type_labels)
radar_detections = pd.read_csv(cleaned_detections_path)

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Using device:", device)

/Users/liuzehan/miniconda3/lib/python3.11/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


Using device: mps


In [2]:
BATCH_SIZE = 32
MAX_LENGTH = 256
HIDDEN_DIM_LSTM = 64
HIDDEN_DIM_LINEAR = 32
NUM_LAYERS_GRU = 1

## Load Data

In [3]:
from dataloader_lstm import load_data

train_loader, val_loader, test_loader = load_data(radar_detections, ais_type_labels, BATCH_SIZE)

-- Summary statistics created from SumStatsBaseline --
-- Summary columns: ['max_speed', 'min_speed', 'avg_speed', 'curviness', 'heading_mean', 'heading_std', 'turning_mean', 'turning_std', 'distance_total', 'distance_o'] --
-- Prepared 14204 track tensors (raw features) on mps --
Data split: 9942 train, 2131 val, 2131 test


In [5]:
import json

with open('model_specs/LSTM_specs.json', 'r') as f:
    specs = json.load(f)

feature_cols = specs['feature_cols']
summary_cols = specs['summary_cols']


## Model Building

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
from core.DICT import TYPE_to_LABEL 

class TypeLSTMClassifier(nn.Module):
    
    def __init__(self, hidden_dim_lstm, hidden_dim_linear, num_layers, features_cols = feature_cols, summary_cols = summary_cols):
        super().__init__()
        self.input_dim = len(features_cols)
        self.summary_dim = len(summary_cols)
        self.TYPE_to_LABEL = TYPE_to_LABEL
        self.num_classes = len(self.TYPE_to_LABEL)

        self.lstm = nn.LSTM(self.input_dim, hidden_dim_lstm, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_dim_lstm + self.summary_dim, hidden_dim_linear)
        self.fc2 = nn.Linear(hidden_dim_linear, self.num_classes)

    def forward(self, x, lengths, summaries):
        # lengths: shape [B], indicating real length of each sequence
        x_packed = nn.utils.rnn.pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        _, (hidden, _) = self.lstm(x_packed)  # LSTM returns (output, (hidden, cell))

        last_hidden = hidden[-1]  #B, hidden_dim_lstm
        concat_layer = torch.cat((last_hidden, summaries), dim=1)  # B, hidden_dim_lstm + summary_dim
        fc1 = self.fc1(concat_layer) #B, hidden_dim_linear
        fc1 = F.relu(fc1)
        logits = self.fc2(fc1) #B, num_classes

        return logits


In [10]:
EPOCH = 5

from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR

model = TypeLSTMClassifier(HIDDEN_DIM_LSTM, HIDDEN_DIM_LINEAR, num_layers=NUM_LAYERS_GRU).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
scheduler = StepLR(optimizer, step_size=2, gamma=0.5) 

train_losses = []
val_losses = []

for epoch in range(EPOCH):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}", leave=False)

    for x_batch, lengths, summaries, y_batch in progress_bar:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        summaries = summaries.to(device)
        
        optimizer.zero_grad()
        logits = model(x_batch, lengths, summaries) #B * num_classes
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    scheduler.step()

    avg_train_loss = total_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Evaluate on validation set
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for x_val, lengths_val, summaries_val, y_val in val_loader:
            x_val = x_val.to(device)
            y_val = y_val.to(device)
            summaries_val = summaries_val.to(device)
            output_val = model(x_val, lengths_val, summaries_val)
            loss_val = criterion(output_val, y_val)
            val_loss += loss_val.item()

    avg_val_loss = val_loss / len(val_loader)
    val_losses.append(avg_val_loss)

    print(f" Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

save_path = 'models/lstm_5_epochs.pth'

torch.save({
    'epoch': epoch + 1,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'train_losses': train_losses,
    'val_losses': val_losses
}, save_path)


 Epoch 1 | Train Loss: 1.5118 | Val Loss: 1.4084


 Epoch 2 | Train Loss: 1.3955 | Val Loss: 1.3854


 Epoch 3 | Train Loss: 1.3665 | Val Loss: 1.3737


 Epoch 4 | Train Loss: 1.3584 | Val Loss: 1.3811


 Epoch 5 | Train Loss: 1.3421 | Val Loss: 1.3804


### Resume Training

In [ ]:
##TODO: UPDATE MODEL 

checkpoint = torch.load('models/gru_5_epochs.pth', map_location=device)

# Recreate the model architecture
model = VesselRNNClassifier(
    input_dim=checkpoint['input_dim'],
    hidden_dim=checkpoint['hidden_dim'],
    num_classes=checkpoint['num_classes'],
    num_layers=NUM_LAYERS_GRU
).to(device)

# Load saved state
model.load_state_dict(checkpoint['model_state_dict'])

# Restore optimizer and scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)  # same LR as before
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Continue tracking losses
TRAIN_LOSSES = checkpoint['train_losses']
VAL_LOSSES = checkpoint['val_losses']

# Resume from this epoch
start_epoch = checkpoint['epoch']

for epoch in range(start_epoch, start_epoch + 5):  # continue 5 more epochs
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}", leave=False)

    for x_batch, lengths, y_batch in progress_bar:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        optimizer.zero_grad()
        logits = model(x_batch, lengths)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    scheduler.step()

    avg_train_loss = total_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Evaluate on validation set
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for x_val, lengths_val, y_val in val_loader:
            output_val = model(x_val, lengths_val)
            loss_val = criterion(output_val, y_val)
            val_loss += loss_val.item()

    avg_val_loss = val_loss / len(val_loader)
    val_losses.append(avg_val_loss)

    print(f"✅ Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")


✅ Epoch 6 | Train Loss: 1.6352 | Val Loss: 1.6514


✅ Epoch 7 | Train Loss: 1.6380 | Val Loss: 1.6324


✅ Epoch 8 | Train Loss: 1.6363 | Val Loss: 1.6632


✅ Epoch 9 | Train Loss: 1.6342 | Val Loss: 1.6847


✅ Epoch 10 | Train Loss: 1.6393 | Val Loss: 1.6743


...

✅ Epoch 36 | Train Loss: 1.6207 | Val Loss: 1.6785


✅ Epoch 37 | Train Loss: 1.6098 | Val Loss: 1.6317


✅ Epoch 38 | Train Loss: 1.6064 | Val Loss: 1.5775


✅ Epoch 39 | Train Loss: 1.5869 | Val Loss: 1.6311


✅ Epoch 40 | Train Loss: 1.5973 | Val Loss: 1.6602


In [55]:
##TODO: UPDATE MODEL PATH

model_path = 'models/gru_40_epochs.pth'

torch.save({
    'epoch': epoch + 1,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'train_losses': train_losses,
    'val_losses': val_losses,
    'input_dim': input_dim,
    'hidden_dim': HIDDEN_DIM,
    'num_classes': num_classes,
}, model_path)


### Model Eval

In [18]:
# Ensure model is in eval mode
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():

    progress_bar = tqdm(test_loader, desc=f"Running Test", leave=False)
    for x_test, lengths_test, summaries_test, y_test in progress_bar:
        x_test = x_test.to(device)
        lengths_test = lengths_test  # assuming these are CPU-side already
        logits = model(x_test, lengths_test, summaries_test)

        preds = torch.argmax(logits, dim=1)  # get class with highest logit
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(y_test.cpu().numpy())  # true labels

# Convert to numpy arrays if needed
import numpy as np
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# (Optional) Evaluate accuracy or confusion matrix
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("🔍 Test Accuracy:", accuracy_score(all_labels, all_preds))
print("\n📊 Classification Report:\n", classification_report(all_labels, all_preds))


🔍 Test Accuracy: 0.5371402042711235

📊 Classification Report:
               precision    recall  f1-score   support

           0       0.49      0.23      0.32       158
           1       0.00      0.00      0.00        25
           2       0.67      0.92      0.78       869
           3       0.74      0.05      0.10       387
           4       0.25      0.05      0.09        39
           5       0.35      0.78      0.48       376
           6       0.18      0.01      0.02       224
           7       0.00      0.00      0.00        76

    accuracy                           0.54      2154
   macro avg       0.34      0.26      0.22      2154
weighted avg       0.52      0.54      0.44      2154



/Users/liuzehan/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/liuzehan/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/liuzehan/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
